# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
#!pip install gmaps
import gmaps
import os
import json
# Import API key
from api_keys import g_key




In [3]:
### Store Part I results into DataFrame
cities_df = pd.read_csv("output_files/WeatherByCities_20200621.csv")
cities_df.head()

,City,Country,Date,Latitude,Longitude,Humidity,Pressure,Temp,Max_Temp,Cloudiness,Wind_Speed
0,East London,ZA,1592799964,-33.02,27.91,76,1024,53.60,53.60,17,8.05
1,Longyearbyen,SJ,1592799947,78.22,15.64,65,1018,44.60,44.60,0,3.36
2,Port Alfred,ZA,1592799911,-33.59,26.89,89,1022,57.00,57.00,23,5.01
3,Nizhniy Tsasuchey,RU,1592800367,50.52,115.13,33,1005,74.75,74.75,57,10.20
4,Touros,BR,1592799902,-5.20,-35.46,94,1014,73.40,73.40,3,5.82


In [4]:
### Humidity Heatmap
gmaps.configure(api_key = g_key)
fig=gmaps.figure()

figure_layout = {'width' : '1000px', 'height' : '600px', 'border' : '1px solid black', 'padding' : '1px' }
locations = cities_df[['Latitude','Longitude']]
weights = cities_df['Humidity']
center_coordinates = (30.3333,31.2333)
fig = gmaps.figure(center = center_coordinates, map_type = "SATELLITE", zoom_level = 1.8, layout=figure_layout)



heatmap_layer = gmaps.heatmap_layer(locations,weights = weights, dissipating=False, max_intensity = 100, point_radius = 3)

fig.add_layer(heatmap_layer)
fig 

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

In [5]:
### Create new DataFrame fitting weather criteria

Ideal_Cities = cities_df[(cities_df['Max_Temp'] >= 70) &
                         (cities_df['Max_Temp'] <= 80) &
                         (cities_df['Wind_Speed'] < 10) &
                         (cities_df['Cloudiness'] == 0)]
Ideal_Cities = Ideal_Cities[['City', 'Country', 'Latitude', 'Longitude']]
Ideal_Cities


,City,Country,Latitude,Longitude
95,Ierapetra,GR,35.01,25.74
116,Noumea,NC,-22.28,166.46
346,São João da Barra,BR,-21.64,-41.05
388,Abu Samrah,SY,35.30,37.18
439,Bongandanga,CD,1.50,21.05
454,Yabrūd,SY,33.97,36.66
469,Kumluca,TR,36.37,30.29
472,Tartous,SY,34.89,35.89
501,Tripoli,LY,32.88,13.19
550,Mont-Dore,NC,-22.28,166.58


In [6]:
Ideal_Cities["Hotel Name"] = ""
Ideal_Cities["Hotel Address"] = ""
Ideal_Cities.head()

,City,Country,Latitude,Longitude,Hotel Name,Hotel Address
95,Ierapetra,GR,35.01,25.74,,
116,Noumea,NC,-22.28,166.46,,
346,São João da Barra,BR,-21.64,-41.05,,
388,Abu Samrah,SY,35.30,37.18,,
439,Bongandanga,CD,1.50,21.05,,


In [7]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}


# Use the lat/lng we recovered to identify airports
for index, row in Ideal_Cities.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        Ideal_Cities.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        Ideal_Cities.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.0079873,
                    "lng": 25.7567464
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.00936272989272,
                        "lng": 25.75801872989272
                    },
                    "southwest": {
                        "lat": 35.00666307010728,
                        "lng": 25.75531907010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "a7d024f49d969b6f8fccbe5494bc583bf2c929c7",
            "name": "Tylissos Beach Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1097,
                 

In [8]:

Ideal_Cities["Hotel Name"].replace('',np.nan,inplace=True)
Final_Ideal_Cities = Ideal_Cities.dropna(how="any")
Final_Ideal_Cities

,City,Country,Latitude,Longitude,Hotel Name,Hotel Address
95,Ierapetra,GR,35.01,25.74,Tylissos Beach Hotel,"City Beach Road Ierapetra to Sitia, Ierapetra"
116,Noumea,NC,-22.28,166.46,Nouvata,"123 Promenade Roger Laroque, Anse Vata, Nouméa"
346,São João da Barra,BR,-21.64,-41.05,Pousada Kactus,"R. Q - Chapéu do Sol, São João da Barra"
469,Kumluca,TR,36.37,30.29,TurkuazKöy,"Orta Mah. Barbaros Caddesi No:119 Finke 1, Finike"
472,Tartous,SY,34.89,35.89,Shahin Tower Hotel,Tartus‎
501,Tripoli,LY,32.88,13.19,Radisson Blu Al Mahary Hotel,"Al Fatah St, The Corniche, Tripoli"
550,Mont-Dore,NC,-22.28,166.58,Hotel Restaurant le Vallon Dore,Mont-Dore
562,Caborca,MX,30.58,-111.10,Motel Elba,"Blvrd Guillermo Padrés 201, La Loma, Santa Ana"
581,Florianópolis,BR,-27.61,-48.50,Majestic Palace Hotel,"Av. Jorn. Rubéns de Arruda Ramos, 2746 - Centr..."


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Final_Ideal_Cities.iterrows()]
locations = Final_Ideal_Cities[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info, display_info_box=False)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

In [18]:
!pip install tabulate

print(Final_Ideal_Cities.to_markdown())

|     | City              | Country   |   Latitude |   Longitude | Hotel Name                      | Hotel Address                                                  |
|----:|:------------------|:----------|-----------:|------------:|:--------------------------------|:---------------------------------------------------------------|
|  95 | Ierapetra         | GR        |      35.01 |       25.74 | Tylissos Beach Hotel            | City Beach Road Ierapetra to Sitia, Ierapetra                  |
| 116 | Noumea            | NC        |     -22.28 |      166.46 | Nouvata                         | 123 Promenade Roger Laroque, Anse Vata, Nouméa                 |
| 346 | São João da Barra | BR        |     -21.64 |      -41.05 | Pousada Kactus                  | R. Q - Chapéu do Sol, São João da Barra                        |
| 469 | Kumluca           | TR        |      36.37 |       30.29 | TurkuazKöy                      | Orta Mah. Barbaros Caddesi No:119 Finke 1, Finike              |
| 47